In [42]:
import os
from glob import glob
from PIL import Image
import numpy as np
import json
import random
from copy import deepcopy
from tqdm import tqdm

In [43]:
coco_json_template={
    "info": {
        "description": "IHC Biomarker Dataset",
        "url": "http://cocodataset.org",
        "version": "1.0",
        "year": 2025,
        "contributor": "Research Team",
        "date_created": "2025/09/29"
    },
    "licenses": [
        {
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }
    ],
    
    "categories": [
        {
            "id": 1,
            "name": "pd-l1 negative tumor cell",  # T Cell Positive
            "supercategory": "biomarker"
        },
        {
            "id": 2,
            "name": "pd-l1 positive tumor cell",  # Programmed Death-Ligand 1
            "supercategory": "biomarker"
        },
        {
            "id": 3,
            "name": "non-tumor cell",  # Programmed Death-Ligand 1
            "supercategory": "biomarker"
        }
    ],
    "images": [],
    "annotations": []
}
def custom_coco_json_creation(json_data,image_id,annotation_id):
    image_info = {
        "id": image_id,
        "width": json_data['image']['width'],
        "height": json_data['image']['height'],
        "file_name": json_data['image']['file_name'],
        "license": 1,
    }
    annotaion_info=[]
    for j in range(len(json_data['annotations'])):
        annotaion_info.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": json_data['annotations'][j]['category_id'],
                "bbox":json_data['annotations'][j]['bbox'],  # [x, y, width, height]
                "iscrowd": 0
            })
        annotation_id+=1
    
    return image_info, annotaion_info

In [40]:
json_list=glob('../../data/IGNITE/annotations/pdl1/individual/*.json')
file_path='../../data/IGNITE/images/pdl1/nuclei/'
train_image_id=1  
val_image_id=1
test_image_id=1
train_annotation_id=1
val_annotation_id=1
test_annotation_id=1
train_json=deepcopy(coco_json_template)
val_json=deepcopy(coco_json_template)
test_json=deepcopy(coco_json_template)
for i in tqdm(range(len(json_list))):

    dataset_classification=random.randint(0,10)
    with open(json_list[i],'r') as f:
        data=json.load(f)
    if dataset_classification<8:
        image_info, annotaion_info=custom_coco_json_creation(data,train_image_id,train_annotation_id)
        train_json['images'].append(image_info)
        train_json['annotations'].extend(annotaion_info)
        train_image_id+=1
        train_annotation_id+=len(annotaion_info)
    elif dataset_classification==8:
        image_info, annotaion_info=custom_coco_json_creation(data,val_image_id,val_annotation_id)
        val_json['images'].append(image_info)
        val_json['annotations'].extend(annotaion_info)
        val_image_id+=1
        val_annotation_id+=len(annotaion_info)
    else:
        image_info, annotaion_info=custom_coco_json_creation(data,test_image_id,test_annotation_id)
        test_json['images'].append(image_info)
        test_json['annotations'].extend(annotaion_info)
        test_image_id+=1
        test_annotation_id+=len(annotaion_info)

100%|██████████| 344/344 [00:03<00:00, 87.09it/s] 
